In [1]:
import time
import requests
import re
from random import randint

import functools

### Первые три декоратора и результат их работы, аналогично с приведённым приером

In [2]:
def benchmark(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'Время выполнения функции {func.__name__}: {end-start:.6f} секунд')
        return result
    return wrapper

def logging(func):
    def wrapper(*args, **kwargs):
        print("Функция вызвана с параметрами:", args, kwargs)
        return func(*args, **kwargs)
    return wrapper

def counter(func):
    def wrapper(*args, **kwargs):
        wrapper.call_count += 1
        print(f"Функция была вызвана: {wrapper.call_count} раз")
        return func(*args, **kwargs)

    wrapper.call_count = 0
    return wrapper

In [3]:
@benchmark
@logging
@counter
def word_count(word, url='https://www.gutenberg.org/files/2638/2638-0.txt'):
    
    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()
    
    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Функция вызвана с параметрами: ('whole',) {}
Функция была вызвана: 1 раз
Время выполнения функции wrapper: 2.337544 секунд
Cлово whole встречается 176 раз


### Декоратор memo и пример его работы на расчёте 42 числа Фибоначчи (больше брать страшно, мой калькулятор на рекурсии долго сидит)

In [4]:
def memo(func):
    mem = dict()
    def wrapper(*args):
        if args not in mem:
            mem[args] = func(*args)
        return mem[args]

    return wrapper

In [6]:
%%time

def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
fibonacci(42)

CPU times: total: 55.7 s
Wall time: 55.7 s


267914296

In [7]:
%%time

@memo
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
fibonacci(42)

CPU times: total: 0 ns
Wall time: 0 ns


267914296